# Introduction
Inspired by predicting sales of things dependent of multivariate observations (eg day of week, holidays) and so on

## Prepare data
Before using deflate train and test data that has been downloaded before (kaggle competitions download -c rossmann-store-sales).

```
unzip -d ./input/rossmann-store-sales.zip ./input/
```

## Further literature
- [kaggle competition](https://www.kaggle.com/c/rossmann-store-sales)
- https://www.mdpi.com/2306-5729/4/1/15
- http://cs229.stanford.edu/proj2015/205_report.pdf


# Preprocessing
Feature engineering, create categorial columns

In [2]:
import pandas as pd
from pandas import DataFrame

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df_train: DataFrame = pd.read_csv('./input/train.csv', parse_dates=['Date'], dtype={'StateHoliday': str, 'Open': bool, 'Promo': bool, 'SchoolHoliday': bool})

# FEATURE ENGINEERING
## month
df_train['Month'] = pd.DatetimeIndex(df_train['Date']).to_period('M')
df_train['Year'] = pd.DatetimeIndex(df_train['Date']).to_period('Y')
df_train['Month_str'] = df_train['Month'].map(lambda month_period:month_period.strftime('%Y-%m'))

## StateHoliday -> categorial
df_train['StateHoliday'] = df_train['StateHoliday'].astype('category')

## StateHoliday -> categorial
df_train['DayOfWeek'] = df_train['DayOfWeek'].astype('category')

df_train.info()
df_train.head(5)



# ==== MONTHLY sales =====
s_sales_monthly: pd.Series = df_train.groupby(by=['Store', 'Month']).Sales.sum()
df_sales_monthly = s_sales_monthly.reset_index(drop=False) # Sets columns of MUltiIndex as columns of
df_sales_monthly['Month_str'] = df_sales_monthly['Month'].map(lambda month_period:month_period.strftime('%Y-%m'))
df_sales_monthly['Year'] = df_sales_monthly['Month'].map(lambda month_period:month_period.strftime('%Y')).astype('int64')
# dataframe with drop=False
df_sales_monthly.info()
df_sales_monthly.head(10)






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  category      
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  bool          
 6   Promo          1017209 non-null  bool          
 7   StateHoliday   1017209 non-null  category      
 8   SchoolHoliday  1017209 non-null  bool          
 9   Month          1017209 non-null  period[M]     
 10  Year           1017209 non-null  period[A-DEC] 
 11  Month_str      1017209 non-null  object        
dtypes: bool(3), category(2), datetime64[ns](1), int64(3), object(1), period[A-DEC](1), period[M](1)
memory usage: 59.2+ MB
<class 'pandas.core.f

,Store,Month,Sales,Month_str,Year
0,1,2013-01,128431,2013-01,2013
1,1,2013-02,125271,2013-02,2013
2,1,2013-03,145169,2013-03,2013
3,1,2013-04,118009,2013-04,2013
4,1,2013-05,118225,2013-05,2013
5,1,2013-06,113411,2013-06,2013
6,1,2013-07,128032,2013-07,2013
7,1,2013-08,118320,2013-08,2013
8,1,2013-09,108867,2013-09,2013
9,1,2013-10,114641,2013-10,2013


Plot monthly sales per Store

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Count of month per Store
#print(df_sales_monthly.groupby(by=['Store']).count())

df_sales_monthly_stores_range = df_sales_monthly[(df_sales_monthly['Store'] < 6) & (df_sales_monthly['Store'] > 0)]

# First n stores
df_train_stores_range: DataFrame = df_train[(df_train['Store'] < 6) & (df_train['Store'] > 0)]
#df_train_stores_5.head(10)

# Monthly sales hue'ed by n Stores for one year
sns.relplot(data=df_sales_monthly_stores_range[df_sales_monthly_stores_range['Year'] == 2014], x="Month_str", y="Sales", hue="Store", kind='line', height=20.0, aspect=1.5)
plt.show()